In [ ]:
import cv2
import time
import datetime
import imutils
import numpy as np

def object_detection():
    input_video = cv2.VideoCapture(0) 
    time.sleep(2)
    while True:
        frames = input_video.read()[1] 
        display = 'Not Moving'
        if input_video.isOpened():
            pel, frames = input_video.read()
        else:
            pel = False

        pel, frame1 = input_video.read()
        pel, frame2 = input_video.read()
        d = cv2.absdiff(frame1, frame2)
        k = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
        ss = cv2.GaussianBlur(k, (7, 7), 0)
        pel, zh = cv2.threshold( ss, 25, 255, cv2.THRESH_BINARY)
        Dilate_frame = cv2.dilate(zh, np.ones((7, 7), np.uint8), iterations=2 )
        f = cv2.erode(Dilate_frame, np.ones((7, 7), np.uint8), iterations=2 )
        img, c, h = cv2.findContours(f, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(frame1, c, -1, (0, 0, 255), 3)
        
        tsv=cv2.cvtColor(frames,cv2.COLOR_BGR2HSV)
        
        low=np.array([80,50,50])
        high=np.array([140,255,255])
        
        low1=np.array([40,50,50])
        high1=np.array([80,255,255])
        
        image_mask=cv2.inRange(tsv,low,high)
        output=cv2.bitwise_and(frames, frames, mask=image_mask)
        
        image_mask1=cv2.inRange(tsv,low1,high1)
        output1=cv2.bitwise_and(frames, frames, mask=image_mask1)
        cv2.imshow("Motion Detection", frame1)
        
        if cv2.waitKey(1) == 27: 
            break

        frame1 = frame2
        pel, frame2 = input_video.read()
        frames = imutils.resize(frames, width=650)
       
        mm = cv2.findContours(Dilate_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
        for c in mm:
            if cv2.contourArea(c) > 50: 
                (x, y, w, h) = cv2.boundingRect(c)  

                cv2.rectangle(frames, (x,y), (x+w, y+h), (0, 255, 0), 3) 
                display = 'Moving'
            else:
                pass
        font = cv2.FONT_HERSHEY_DUPLEX 
        cv2.putText(frames, 'Object Status: %s' % (display), 
            (10,20), cv2.FONT_HERSHEY_DUPLEX , 0.5, (0, 0, 255), 2)
        cv2.putText(frames, datetime.datetime.now().strftime('%A %d %B %Y %I:%M:%S%p'), 
            (10, frames.shape[0] - 10), cv2.FONT_HERSHEY_DUPLEX , 0.35, (0, 0, 255),1) 
        cv2.imshow('Main Frame', frames)
        cv2.imshow("Blue Colour Objects",output)
        cv2.imshow("Green Colour Objects ",output1)
        key = cv2.waitKey(1) & 0xFF 
        if key == ord('q'):
            cv2.destroyAllWindows()
            break
                
if __name__=='__main__':    
    object_detection()











